### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from configuration import Config

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### View your sample population

In [5]:
sample_pop = configs.params['enc_type'].initialize()
print(sample_pop)

[{'gene': array([14, 18, 68, 60, 51, 56, 11, 41, 33, 82, 58, 19, 84, 96, 27, 50, 97,
        54, 25, 73, 45,  0, 88, 42, 38, 77, 23, 26, 61,  1, 72, 37, 80, 94,
         6, 86, 28, 70, 83, 49, 62, 89, 78, 20, 47, 74,  3,  2, 17, 36, 46,
        15,  5, 48, 22, 21, 40, 95, 66, 55, 31,  7, 32, 79, 12, 57, 81, 13,
        69, 53, 34, 75, 24, 35, 29, 67,  4, 92, 39, 10, 16, 76, 87, 64, 90,
        91,  8, 30, 63, 43, 99, 44, 59, 65, 98, 71, 93,  9, 52, 85]), 'fitness': array([0]), 'meta': None}
 {'gene': array([50, 79, 18, 20, 17, 14, 43, 37, 57, 94, 21, 44, 93, 22, 91,  0, 69,
        23, 51, 74, 13, 48, 66, 49, 33, 81, 15, 68, 87, 11,  8, 32, 90, 45,
        72, 38, 78, 19, 67, 99, 39, 63, 85, 35, 62, 84, 58, 34, 70, 60, 96,
        41, 59,  1, 76,  6,  2, 42, 64, 65, 36,  5,  4, 54, 86, 89, 97, 88,
        29, 31, 95, 77, 25, 46, 12, 10, 28, 73, 82, 98, 16, 26, 75,  7,  9,
        61, 80, 53, 83, 55, 24, 56, 27,  3, 71, 47, 30, 52, 92, 40]), 'fitness': array([0]), 'meta': None}
 {'gene'

### Define your evaluation function

Your evaluation function should update all fitness values of an individual ```ind```. For example, the evaluation function for the single-objective Traveling Salesman Problem is defined below.

```
def eval(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() + dist[ind['gene'][-1]][0]

    return pop
```

In [6]:
def TSP(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.params['gene_size'])]).sum() + dist[ind['gene'][-1]][0]

    return pop

In [7]:
sample_ind = TSP(sample_pop)
print(sample_ind[0])

{'gene': array([14, 18, 68, 60, 51, 56, 11, 41, 33, 82, 58, 19, 84, 96, 27, 50, 97,
       54, 25, 73, 45,  0, 88, 42, 38, 77, 23, 26, 61,  1, 72, 37, 80, 94,
        6, 86, 28, 70, 83, 49, 62, 89, 78, 20, 47, 74,  3,  2, 17, 36, 46,
       15,  5, 48, 22, 21, 40, 95, 66, 55, 31,  7, 32, 79, 12, 57, 81, 13,
       69, 53, 34, 75, 24, 35, 29, 67,  4, 92, 39, 10, 16, 76, 87, 64, 90,
       91,  8, 30, 63, 43, 99, 44, 59, 65, 98, 71, 93,  9, 52, 85]), 'fitness': array([53087]), 'meta': None}


### Define a custom Plotly figure to display an individual's phenotypic (high-level) representation

For example, the phenotypic representation for a solution Traveling Salesman Problem is defined below as a graph network

```
def custom(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig
```

In [8]:
def network(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.params['gene_size'])] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.params['gene_size'])] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig

### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [9]:
configs.params['cust_eval'], configs.params['cust_vis'] = TSP, network

### Run the application

In [10]:
main_app = app_file.create_app(configs)
main_app.run_server()

Dash app running on http://127.0.0.1:8050/


### Testing new Techniques

In [11]:
from single import Single
from permutation import Perm
configs.params['sel_type'] = Single(configs.params).tournament
configs.params['rep_type'] = Single(configs.params).tournament
configs.params['tourn_size'] = 16
configs.params['enc_type'].params['rec_type'] = configs.params['enc_type'].Cross(configs.params).PMX